# Automated Machine Learning (AutoML) Search

## Background

### Machine Learning

[Machine learning](https://en.wikipedia.org/wiki/Machine_learning) (ML) is the process of constructing a mathematical model of a system based on a sample dataset collected from that system.

One of the main goals of training an ML model is to teach the model to separate the signal present in the data from the noise inherent in system and in the data collection process. If this is done effectively, the model can then be used to make accurate predictions about the system when presented with new, similar data. Additionally, introspecting on an ML model can reveal key information about the system being modeled, such as which inputs and transformations of the inputs are most useful to the ML model for learning the signal in the data, and are therefore the most predictive.

There are [a variety](https://en.wikipedia.org/wiki/Machine_learning#Approaches) of ML problem types. Supervised learning describes the case where the collected data contains an output value to be modeled and a set of inputs with which to train the model. EvalML focuses on training supervised learning models.

EvalML supports three common supervised ML problem types. The first is regression, where the target value to model is a continuous numeric value. Next are binary and multiclass classification, where the target value to model consists of two or more discrete values or categories. The choice of which supervised ML problem type is most appropriate depends on domain expertise and on how the model will be evaluated and used.


### AutoML and Search

[AutoML](https://en.wikipedia.org/wiki/Automated_machine_learning) is the process of automating the construction, training and evaluation of ML models. Given a data and some configuration, AutoML searches for the most effective and accurate ML model or models to fit the dataset. During the search, AutoML will explore different combinations of model type, model parameters and model architecture.

An effective AutoML solution offers several advantages over constructing and tuning ML models by hand. AutoML can assist with many of the difficult aspects of ML, such as avoiding overfitting and underfitting, imbalanced data, detecting data leakage and other potential issues with the problem setup, and automatically applying best-practice data cleaning, feature engineering, feature selection and various modeling techniques. AutoML can also leverage search algorithms to optimally sweep the hyperparameter search space, resulting in model performance which would be difficult to achieve by manual training.

## AutoML in EvalML

EvalML supports all of the above and more.

In its simplest usage, the AutoML search interface requires only the input data, the target data and a `problem_type` specifying what kind of supervised ML problem to model.

** Graphing methods, like AutoMLSearch, on Jupyter Notebook and Jupyter Lab require [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/user_install.html) to be installed.

** If graphing on Jupyter Lab, [jupyterlab-plotly](https://plotly.com/python/getting-started/#jupyterlab-support-python-35) required. To download this, make sure you have [npm](https://nodejs.org/en/download/) installed.

To provide data to EvalML, it is recommended that you create a `DataTable` object using [the Woodwork project](https://woodwork.alteryx.com/en/stable/).

EvalML also accepts and works well with pandas `DataFrames`. But using the `DataTable` makes it easy to control how EvalML will treat each feature, as a numeric feature, a categorical feature, a text feature or other type of feature. Woodwork's `DataTable` includes features like inferring when a categorical feature should be treated as a text feature. For this reason, if you don't provide Woodwork objects, EvalML will raise a warning.

In [1]:
import evalml

X, y = evalml.demos.load_breast_cancer()

import woodwork as ww
X_dt = ww.DataTable(X)
y_dc = ww.DataColumn(y)

automl = evalml.automl.AutoMLSearch(problem_type='binary')
automl.search(X_dt, y_dc)

2020-11-05 12:40:48,010 featuretools - WARNING    Featuretools failed to load plugin nlp_primitives from library nlp_primitives. For a full stack trace, set logging to debug.
Using default limit of max_iterations=5.

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Searching up to 5 pipelines. 
Allowed model families: decision_tree, random_forest, xgboost, catboost, linear_model, extra_trees, lightgbm



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/5) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 12.868
2/5) Decision Tree Classifier w/ Imputer      Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 1.965
High coefficient of variation (cv >= 0.2) within cross validation scores. Decision Tree Classifier w/ Imputer may not perform as estimated on unseen data.
3/5) LightGBM Classifier w/ Imputer           Elapsed:00:00
	Starting cross validation
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM

The AutoML search will log its progress, reporting each pipeline and parameter set evaluated during the search.

There are a number of mechanisms to control the AutoML search time. One way is to set the maximum number of candidate models to be evaluated during AutoML using `max_iterations`. By default, AutoML will search a fixed number of iterations and parameter pairs (`max_iterations=5`). The first pipeline to be evaluated will always be a baseline model representing a trivial solution. 

The AutoML interface supports a variety of other parameters. For a comprehensive list, please [refer to the API reference.](../generated/evalml.automl.AutoMLSearch.ipynb)

### Detecting Problem Type

EvalML includes a simple method, `detect_problem_type`, to help determine the problem type given the target data. 

This function can return the predicted problem type as a ProblemType enum, choosing from ProblemType.BINARY, ProblemType.MULTICLASS, and ProblemType.REGRESSION. If the target data is invalid (for instance when there is only 1 unique label), the function will throw an error instead.


In [2]:
import pandas as pd
from evalml.problem_types import detect_problem_type

y = pd.Series([0, 1, 1, 0, 1, 1])
detect_problem_type(y)

<ProblemTypes.BINARY: 'binary'>

### Objective parameter

AutoMLSearch takes in an `objective` parameter to determine which `objective` to optimize for. By default, this parameter is set to `auto`, which allows AutoML to choose `LogLossBinary` for binary classification problems, `LogLossMulticlass` for multiclass classification problems, and `R2` for regression problems.

It should be noted that the `objective` parameter is only used in ranking and helping choose the pipelines to iterate over, but is not used to optimize each individual pipeline during fit-time.

To get the default objective for each problem type, you can use the `get_default_primary_search_objective` function.

In [3]:
from evalml.automl import get_default_primary_search_objective

binary_objective = get_default_primary_search_objective("binary")
multiclass_objective = get_default_primary_search_objective("multiclass")
regression_objective = get_default_primary_search_objective("regression")

print(binary_objective.name)
print(multiclass_objective.name)
print(regression_objective.name)

Log Loss Binary
Log Loss Multiclass
R2


### Data Checks

`AutoMLSearch.search` runs a set of data checks before beginning the search process to ensure that the input data being passed will not run into some common issues before running a potentially time-consuming search. If the data checks find any potential errors, an exception will be thrown before the search begins, allowing users to inspect their data to avoid confusing errors that may arise later during the search process.

This behavior is controlled by the `data_checks` parameter which can take in either a `DataChecks` object, a list of `DataCheck` objects, `None`, or valid string inputs (`"disabled"`, `"auto"`). By default, this parameter is set to `auto`, which runs the default collection of data sets defined in the `DefaultDataChecks` class. If set to `"disabled"` or None, no data checks will run.

### Using custom pipelines

EvalML's AutoML algorithm generates a set of pipelines to search with. To provide a custom set instead, set allowed_pipelines to a list of [custom pipeline](pipelines.ipynb) classes. Note: this will prevent AutoML from generating other pipelines to search over.

In [6]:
from evalml.pipelines import MulticlassClassificationPipeline

class CustomMulticlassClassificationPipeline(MulticlassClassificationPipeline):
    component_graph = ['Simple Imputer', 'Random Forest Classifier']

automl_custom = evalml.automl.AutoMLSearch(problem_type='multiclass', allowed_pipelines=[CustomMulticlassClassificationPipeline])

Using default limit of max_iterations=5.



### Stopping the search early

To stop the search early, hit `Ctrl-C`. This will bring up a prompt asking for confirmation. Responding with `y` will immediately stop the search. Responding with `n` will continue the search.

![Interrupting Search Demo](keyboard_interrupt_demo_updated.gif)

## View Rankings
A summary of all the pipelines built can be returned as a pandas DataFrame which is sorted by score. The score column contains the average score across all cross-validation folds while the validation_score column is computed from the first cross-validation fold.

In [7]:
automl.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,2,LightGBM Classifier w/ Imputer,0.130426,0.171504,98.986464,True,{'Imputer': {'categorical_impute_strategy': 'm...
1,3,Extra Trees Classifier w/ Imputer,0.146243,0.150788,98.863555,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,4,Elastic Net Classifier w/ Imputer + Standard S...,0.504486,0.505864,96.079663,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,1,Decision Tree Classifier w/ Imputer,1.965214,1.508027,84.728426,True,{'Imputer': {'categorical_impute_strategy': 'm...
4,0,Mode Baseline Binary Classification Pipeline,12.868443,12.906595,0.000000,False,{'Baseline Classifier': {'strategy': 'mode'}}


## Describe Pipeline
Each pipeline is given an `id`. We can get more information about any particular pipeline using that `id`. Here, we will get more information about the pipeline with `id = 1`.

In [8]:
automl.describe_pipeline(1)

***************************************
* Decision Tree Classifier w/ Imputer *
***************************************

Problem Type: binary
Model Family: Decision Tree

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. Decision Tree Classifier
	 * criterion : gini
	 * max_features : auto
	 * max_depth : 6
	 * min_samples_split : 2
	 * min_weight_fraction_leaf : 0.0

Training
Training for binary problems.
Total training time (including CV): 0.2 seconds

Cross Validation
----------------
             Log Loss Binary  MCC Binary   AUC  Precision    F1  Balanced Accuracy Binary  Accuracy Binary # Training # Testing
0                      1.508       0.854 0.936      0.903 0.909                     0.928            0.932    379.000   190.000
1                      2.547       0.843 0.910      0.901 0.901                     0.921            0.926    379.000   190.0

## Get Pipeline
We can get the object of any pipeline via their `id` as well:

In [9]:
pipeline = automl.get_pipeline(1)
print(pipeline.name)
print(pipeline.parameters)

Decision Tree Classifier w/ Imputer
{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Decision Tree Classifier': {'criterion': 'gini', 'max_features': 'auto', 'max_depth': 6, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0}}


### Get best pipeline
If we specifically want to get the best pipeline, there is a convenient accessor for that.

In [10]:
best_pipeline = automl.best_pipeline
print(best_pipeline.name)
print(best_pipeline.parameters)

LightGBM Classifier w/ Imputer
{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'LightGBM Classifier': {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 0, 'num_leaves': 31, 'min_child_samples': 20, 'n_jobs': -1, 'bagging_freq': 0, 'bagging_fraction': 0.9}}


## Limiting the AutoML Search Space
The AutoML search algorithm first trains each component in the pipeline with their default values. After the first iteration, it then tweaks the parameters of these components using the pre-defined hyperparameter ranges that these components have. To limit the search over certain hyperparameter ranges, use `make_pipeline` to define a pipeline with a custom hyperparameter range. Hyperparameter ranges can be found through the [API reference](https://evalml.alteryx.com/en/stable/api_reference.html) for each component. Note: The default value of the component must be included in any specified hyperparameter range for AutoMLSearch to succeed. Additionally, the parameter value must be specified as a list, even for just one value.

In [11]:
from evalml import AutoMLSearch
from evalml.demos import load_fraud
from evalml.pipelines.components.utils import get_estimators
from evalml.model_family import ModelFamily
from evalml.pipelines.utils import make_pipeline
import woodwork as ww

X, y = load_fraud()

# example of setting parameter to just one value
custom_hyperparameters = {'Imputer': {
    'numeric_impute_strategy': ['mean']
}}

# limit the numeric impute strategy to include only `mean` and `median`
# `mean` is the default value for this argument, and it needs to be included in the specified hyperparameter range
custom_hyperparameters = {'Imputer': {
    'numeric_impute_strategy': ['mean', 'median']
}}

# AutoMLSearch uses woodwork, so we want to use ww to convert the appropriate types when making pipelines
# ww changes the original X and y data, so we pass that instead of X_dt, y_dc
X_dt = ww.DataTable(X)
y_dc = ww.DataColumn(y)

estimators = get_estimators('binary', [ModelFamily.EXTRA_TREES])
pipelines_with_custom_hyperparameters = [make_pipeline(X, y, estimator, 'binary', custom_hyperparameters) for estimator in estimators]
automl = AutoMLSearch(problem_type='binary', allowed_pipelines=pipelines_with_custom_hyperparameters)
automl.search(X, y)
automl.best_pipeline.hyperparameters

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992
Targets
False    84.82%
True     15.18%
Name: fraud, dtype: object
Using default limit of max_iterations=5.

`X` passed was not a DataTable. EvalML will try to convert the input as a Woodwork DataTable and types will be inferred. To control this behavior, please pass in a Woodwork DataTable instead.
`y` passed was not a DataColumn. EvalML will try to convert the input as a Woodwork DataTable and types will be inferred. To control this behavior, please pass in a Woodwork DataTable instead.
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Searching up to 5 pipelines. 
Allowed model families: extra_trees



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/5) Mode Baseline Binary Classification P... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 5.243
2/5) Extra Trees Classifier w/ Imputer + D... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.357
3/5) Extra Trees Classifier w/ Imputer + D... Elapsed:00:23
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.378
4/5) Extra Trees Classifier w/ Imputer + D... Elapsed:00:47
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.353
5/5) Extra Trees Classifier w/ Imputer + D... Elapsed:01:26
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.337

Search finished after 01:57            
Best pipeline: Extra Trees Classifier w/ Imputer + DateTime Featurization Component + One Hot Encoder
Best pipeline Log Loss Binary: 0.337176


{'Imputer': {'categorical_impute_strategy': ['most_frequent'],
  'numeric_impute_strategy': ['mean', 'median']},
 'DateTime Featurization Component': {},
 'One Hot Encoder': {},
 'Extra Trees Classifier': {'n_estimators': Integer(low=10, high=1000, prior='uniform', transform='identity'),
  'max_features': ['auto', 'sqrt', 'log2'],
  'max_depth': Integer(low=4, high=10, prior='uniform', transform='identity')}}

## Access raw results

The `AutoMLSearch` class records detailed results information under the `results` field, including information about the cross-validation scoring and parameters.

In [12]:
automl.results

{'pipeline_results': {0: {'id': 0,
   'pipeline_name': 'Mode Baseline Binary Classification Pipeline',
   'pipeline_class': evalml.pipelines.classification.baseline_binary.ModeBaselineBinaryPipeline,
   'pipeline_summary': 'Baseline Classifier',
   'parameters': {'Baseline Classifier': {'strategy': 'mode'}},
   'score': 5.243060307948459,
   'high_variance_cv': False,
   'training_time': 0.9124500751495361,
   'cv_data': [{'all_objective_scores': OrderedDict([('Log Loss Binary',
                   5.243353291477845),
                  ('MCC Binary', 0.0),
                  ('AUC', 0.5),
                  ('Precision', 0.0),
                  ('F1', 0.0),
                  ('Balanced Accuracy Binary', 0.5),
                  ('Accuracy Binary', 0.848189373256128),
                  ('# Training', 66661),
                  ('# Testing', 33331)]),
     'score': 5.243353291477845,
     'binary_classification_threshold': 0.5},
    {'all_objective_scores': OrderedDict([('Log Loss Binary',
  